## Clean/transform game_plays_bronze -> game_plays_silver

##### 1. Import and load table

In [6]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F
from pyspark.sql import functions as F, Window

df = spark.read.format("delta").load("Tables/game_plays_bronze")

#display(df.limit(5))

StatementMeta(, ee6db201-31b8-4408-aba1-0e4240eaa4ec, 8, Finished, Available, Finished)

##### 2. Clean and transform data

In [7]:
# 1. Drop unwanted columns
df = df.drop("st_x", "st_y")

# 2. Remove duplicate rows with exact information
df = df.dropDuplicates()

# 3. Function to convert camelCase / PascalCase to snake_case
def to_snake_case(name):
    return ''.join(['_' + c.lower() if c.isupper() else c for c in name]).lstrip('_')

# Apply to all columns in one line
df = df.toDF(*[to_snake_case(c) for c in df.columns])

"""
# Explicit rename for special case
df = df.withColumnRenamed("periodType", "period_type") \
                   .withColumnRenamed("periodTime", "period_time") \
                   .withColumnRenamed("periodTimeRemaining", "period_time_remaining") \
                   .withColumnRenamed("dateTime", "date_time") \
                   .withColumnRenamed("secondaryType", "secondary_type")
"""

# 4: Cast data types
df = (
    df  # ✅ Start from df, not df_clean
    .withColumn("play_id", F.col("play_id").cast("string"))
    .withColumn("game_id", F.col("game_id").cast("string"))
    .withColumn("team_id_for", F.col("team_id_for").cast("string"))
    .withColumn("team_id_against", F.col("team_id_against").cast("string"))
    .withColumn("x", F.col("x").cast("int"))
    .withColumn("y", F.col("y").cast("int"))
    .withColumn("period", F.col("period").cast("int"))
    .withColumn("period_time", F.col("period_time").cast("int"))
    .withColumn("period_time_remaining", F.col("period_time_remaining").cast("int"))
    .withColumn("goals_away", F.col("goals_away").cast("int"))
    .withColumn("goals_home", F.col("goals_home").cast("int"))
    .withColumn("date_time", F.to_timestamp("date_time", "yyyy/mm/dd H:mm"))
)

# 5: Replace blank strings with nulls (optional)
df = df.select([
    F.when(F.col(c) == "", None).otherwise(F.col(c)).alias(c) if dtype == "string" else F.col(c)
    for c, dtype in df.dtypes
])

# 6: Trim and clean join key to avoid mismatch issues
df = df.withColumn("play_id", F.trim(F.col("play_id")))

"""
# Show results
display(df.limit(10))

rows_final = df.count()
cols_final = len(df.columns)
print("")
print("Number of rows:", rows_final)
print("Number of columns:", cols_final)
"""

StatementMeta(, ee6db201-31b8-4408-aba1-0e4240eaa4ec, 9, Finished, Available, Finished)

'\n# Show results\ndisplay(df.limit(10))\n\nrows_final = df.count()\ncols_final = len(df.columns)\nprint("")\nprint("Number of rows:", rows_final)\nprint("Number of columns:", cols_final)\n'

##### 3. Load data to silver table

In [8]:
# Incoming Bronze dataframe
source_df = df  
key_col = "play_id"

# Path to Silver Lakehouse Delta table
target_path = "abfss://dc478dd4-e53e-4f21-add0-2e376dc173fe@onelake.dfs.fabric.microsoft.com/ce7ef0e9-78af-44db-b5ee-839dcf1c9e98/Tables/games_play_silver"

if DeltaTable.isDeltaTable(spark, target_path):
    # Load existing target table
    existing_df = spark.read.format("delta").load(target_path).select(key_col).distinct()

    # Keep only new keys
    new_rows_df = source_df.join(existing_df, on=key_col, how="left_anti")

    if new_rows_df.limit(1).count() > 0:
        (new_rows_df.write
            .format("delta")
            .mode("append")
            .save(target_path))
        print(f"✅ Appended {new_rows_df.count()} new rows to games_play_silver in Lakehouse_Silver.")
    else:
        print("No new rows to append. games_play_silver is already up to date.")
else:
    # First load → create the Silver table
    (source_df.write
        .format("delta")
        .mode("overwrite")
        .save(target_path))
    print(f"✅ Initial load complete: created games_play_silver in Lakehouse_Silver with {source_df.count()} rows.")


StatementMeta(, ee6db201-31b8-4408-aba1-0e4240eaa4ec, 10, Finished, Available, Finished)

No new rows to append. games_play_silver is already up to date.


In [9]:
"""
# 1. Cast data types
df = (
    df  # ✅ Start from df, not df_clean
    .withColumn("play_id", F.col("play_id").cast("string"))
    .withColumn("game_id", F.col("game_id").cast("string"))
    .withColumn("team_id_for", F.col("team_id_for").cast("string"))
    .withColumn("team_id_against", F.col("team_id_against").cast("string"))
    .withColumn("x", F.col("x").cast("int"))
    .withColumn("y", F.col("y").cast("int"))
    .withColumn("st_x", F.col("st_x").cast("int"))
    .withColumn("st_y", F.col("st_y").cast("int"))
    .withColumn("period", F.col("period").cast("int"))
    .withColumn("periodTime", F.col("periodTime").cast("int"))
    .withColumn("periodTimeRemaining", F.col("periodTimeRemaining").cast("int"))
    .withColumn("goals_away", F.col("goals_away").cast("int"))
    .withColumn("goals_home", F.col("goals_home").cast("int"))
    .withColumn("dateTime", F.to_timestamp("dateTime", "yyyy/mm/dd H:mm"))
)


#display(df.limit(5))
# 2. Remove duplicate rows
df = df.dropDuplicates()
# Check for exact duplicate rows
#dup_count = df.groupBy(df.columns).count().filter(F.col("count") > 1).count()
#print("Number of exact duplicated rows:", dup_count)
#print("")

# 4. Check schema data type
#df.printSchema()
#display(df.limit(5))
#df_clean.show()
# Step 1: Drop unwanted columns
df = df.drop("st_x", "st_y")

# Step 2: Rename columns and convert to snake_case (your existing logic)
df = df.withColumnRenamed("periodType", "period_type") \
                   .withColumnRenamed("periodTime", "period_time") \
                   .withColumnRenamed("periodTimeRemaining", "period_time_remaining") \
                   .withColumnRenamed("dateTime", "date_time") \
                   .withColumnRenamed("secondaryType", "secondary_type")

def to_snake_case(name):
    return ''.join(['_' + c.lower() if c.isupper() else c for c in name]).lstrip('_')

df = df.toDF(*[to_snake_case(c) for c in df.columns])

# Step 3: Replace blank strings with nulls (optional)
df = df.select([
    F.when(F.col(c) == "", None).otherwise(F.col(c)).alias(c) if dtype == "string" else F.col(c)
    for c, dtype in df.dtypes
])

# Step 4: Trim and clean join key to avoid mismatch issues
df = df.withColumn("play_id", F.trim(F.col("play_id")))
"""


StatementMeta(, ee6db201-31b8-4408-aba1-0e4240eaa4ec, 11, Finished, Available, Finished)

'\n# 1. Cast data types\ndf = (\n    df  # ✅ Start from df, not df_clean\n    .withColumn("play_id", F.col("play_id").cast("string"))\n    .withColumn("game_id", F.col("game_id").cast("string"))\n    .withColumn("team_id_for", F.col("team_id_for").cast("string"))\n    .withColumn("team_id_against", F.col("team_id_against").cast("string"))\n    .withColumn("x", F.col("x").cast("int"))\n    .withColumn("y", F.col("y").cast("int"))\n    .withColumn("st_x", F.col("st_x").cast("int"))\n    .withColumn("st_y", F.col("st_y").cast("int"))\n    .withColumn("period", F.col("period").cast("int"))\n    .withColumn("periodTime", F.col("periodTime").cast("int"))\n    .withColumn("periodTimeRemaining", F.col("periodTimeRemaining").cast("int"))\n    .withColumn("goals_away", F.col("goals_away").cast("int"))\n    .withColumn("goals_home", F.col("goals_home").cast("int"))\n    .withColumn("dateTime", F.to_timestamp("dateTime", "yyyy/mm/dd H:mm"))\n)\n\n\n#display(df.limit(5))\n# 2. Remove duplicate rows

In [10]:
"""
# Welcome to your new notebook
# 1. Type here in the cell editor to add code!


# 3 . Cast data types
df_clean = (
    df  # ✅ Start from df, not df_clean
    .withColumn("play_id", F.col("play_id").cast("string"))
    .withColumn("game_id", F.col("game_id").cast("string"))
    .withColumn("team_id_for", F.col("team_id_for").cast("string"))
    .withColumn("team_id_against", F.col("team_id_against").cast("string"))
    .withColumn("x", F.col("x").cast("int"))
    .withColumn("y", F.col("y").cast("int"))
    .withColumn("st_x", F.col("st_x").cast("int"))
    .withColumn("st_y", F.col("st_y").cast("int"))
    .withColumn("period", F.col("period").cast("int"))
    .withColumn("periodTime", F.col("periodTime").cast("int"))
    .withColumn("periodTimeRemaining", F.col("periodTimeRemaining").cast("int"))
    .withColumn("goals_away", F.col("goals_away").cast("int"))
    .withColumn("goals_home", F.col("goals_home").cast("int"))
    .withColumn("dateTime", F.to_timestamp("dateTime", "yyyy/mm/dd H:mm"))
)


#display(df.limit(5))
# 2. Remove duplicate rows
df = df.dropDuplicates()
# Check for exact duplicate rows
dup_count = df.groupBy(df.columns).count().filter(F.col("count") > 1).count()
#print("Number of exact duplicated rows:", dup_count)
#print("")

# 4. Check schema data type
df.printSchema()
#display(df.limit(5))
#df_clean.show()
# Step 1: Drop unwanted columns
df_clean = df_clean.drop("st_x", "st_y")

# Step 2: Rename columns and convert to snake_case (your existing logic)
df_clean = df_clean.withColumnRenamed("periodType", "period_type") \
                   .withColumnRenamed("periodTime", "period_time") \
                   .withColumnRenamed("periodTimeRemaining", "period_time_remaining") \
                   .withColumnRenamed("dateTime", "date_time") \
                   .withColumnRenamed("secondaryType", "secondary_type")

def to_snake_case(name):
    return ''.join(['_' + c.lower() if c.isupper() else c for c in name]).lstrip('_')

df_clean = df_clean.toDF(*[to_snake_case(c) for c in df_clean.columns])

# Step 3: Replace blank strings with nulls (optional)
df_clean = df_clean.select([
    F.when(F.col(c) == "", None).otherwise(F.col(c)).alias(c) if dtype == "string" else F.col(c)
    for c, dtype in df_clean.dtypes
])

# Step 4: Trim and clean join key to avoid mismatch issues
df_clean = df_clean.withColumn("play_id", F.trim(F.col("play_id")))
# Step 5: Define Silver Delta table path
target_path = "abfss://dc478dd4-e53e-4f21-add0-2e376dc173fe@onelake.dfs.fabric.microsoft.com/ce7ef0e9-78af-44db-b5ee-839dcf1c9e98/Tables/games_play_silver"

try:
    # Load existing Silver table
    delta_tbl = DeltaTable.forPath(spark, target_path)
    silver_df = delta_tbl.toDF()

    # Align Bronze df schema to Silver df schema by casting columns
    for field in silver_df.schema.fields:
        if field.name in df_clean.columns:
            df_clean = df_clean.withColumn(field.name, F.col(field.name).cast(field.dataType))

    # Debug: Check counts and sample keys
    print(f"Bronze rows count: {df_clean.count()}")
    print(f"Silver rows count: {silver_df.count()}")
    print("Bronze play_id sample:")
    df_clean.select("play_id").distinct().show(5, truncate=False)
    print("Silver play_id sample:")
    silver_df.select("play_id").distinct().show(5, truncate=False)

    # Debug: Check for matching keys
    common_keys_count = df_clean.join(silver_df, "play_id", "inner").count()
    print(f"Number of matching play_id between Bronze and Silver: {common_keys_count}")

    # Debug: Show Bronze values for x,y,period_time_remaining before merge
    print("Sample Bronze data (x, y, period_time_remaining):")
    df_clean.select("play_id", "x", "y", "period_time_remaining").show(10, truncate=False)

    # Perform merge/upsert with explicit F.col()
    delta_tbl.alias("silver").merge(
        df_clean.alias("bronze"),
        "silver.play_id = bronze.play_id"
    ).whenMatchedUpdate(set={
        "x": F.col("bronze.x"),
        "y": F.col("bronze.y"),
        "period_time_remaining": F.col("bronze.period_time_remaining")
    }).whenNotMatchedInsertAll().execute()

    print("✅ Merge (upsert) operation complete: Silver table updated.")

except Exception as e:
    print(f"⚠️ Silver table does not exist or error: {e}")
    # Initial load with schema overwrite to avoid conflicts
    df_clean.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(target_path)
    print(f"✅ Initial load complete: Silver table created with {df_clean.count()} rows.")

# Step 6: Show sample data from Silver after merge
print("Sample data from Silver table after upsert:")
spark.read.format("delta").load(target_path).select("play_id", "x", "y", "period_time_remaining").show(10)
df_clean.filter(col("play_id") == "2017021013_184") \
        .select("play_id", "x", "y", "period_time_remaining") \
        .show(truncate=False)
        # Load the Silver Delta table
silver_path = "abfss://dc478dd4-e53e-4f21-add0-2e376dc173fe@onelake.dfs.fabric.microsoft.com/ce7ef0e9-78af-44db-b5ee-839dcf1c9e98/Tables/games_play_silver"
df_silver = spark.read.format("delta").load(silver_path)

# Filter for a specific play_id and display key columns
df_silver.filter(col("play_id") == "2017021013_184") \
         .select("play_id", "x", "y", "period_time_remaining") \
         .show(truncate=False)

"""

StatementMeta(, ee6db201-31b8-4408-aba1-0e4240eaa4ec, 12, Finished, Available, Finished)

'\n# Welcome to your new notebook\n# 1. Type here in the cell editor to add code!\n\n\n# 3 . Cast data types\ndf_clean = (\n    df  # ✅ Start from df, not df_clean\n    .withColumn("play_id", F.col("play_id").cast("string"))\n    .withColumn("game_id", F.col("game_id").cast("string"))\n    .withColumn("team_id_for", F.col("team_id_for").cast("string"))\n    .withColumn("team_id_against", F.col("team_id_against").cast("string"))\n    .withColumn("x", F.col("x").cast("int"))\n    .withColumn("y", F.col("y").cast("int"))\n    .withColumn("st_x", F.col("st_x").cast("int"))\n    .withColumn("st_y", F.col("st_y").cast("int"))\n    .withColumn("period", F.col("period").cast("int"))\n    .withColumn("periodTime", F.col("periodTime").cast("int"))\n    .withColumn("periodTimeRemaining", F.col("periodTimeRemaining").cast("int"))\n    .withColumn("goals_away", F.col("goals_away").cast("int"))\n    .withColumn("goals_home", F.col("goals_home").cast("int"))\n    .withColumn("dateTime", F.to_timestam